In [ ]:
import smartcitizen_connector
import scdata as sc
import time
now = time.localtime()

# Glasgow Batch #2

Smart Citizen Kit 2.1

## Get devices

In [ ]:
df = smartcitizen_connector.search_by_query(endpoint='devices',
                                            search_items=[{
                                                'key': 'postprocessing_id', 
                                                'value': 'not_null',
                                                'full': True
                                            }])

In [ ]:
devices = []
for item in df['postprocessing']:
    print (item['device_id'], df.loc[item['device_id'], 'name'], item['meta'])
    if item['meta'] == 'delivery-glasgow 2.1-2':
        devices.append(item['device_id'])
#     if item['meta'] == 'delivery-glasgow-ref':
#       devices.append(item['device_id'])

In [ ]:
len(devices)

In [ ]:
test = sc.Test(name=f'GLASGOW_BATCH_2.1_2',
            devices=[sc.Device(blueprint='sc_air', params=sc.APIParams(id=id)) for id in devices],
        force_recreate = True)
test.options.cache=True

## Load

In [ ]:
test = sc.Test(name=f'GLASGOW_BATCH_2.1_2')

In [ ]:
await test.load()

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "TEMP",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'Temperature', 
                            'ylabel': {1: 'Temperature (degC)'}
                           }, 
              options = {'frequency': '5Min', 'min_date': '2024-07-10'})

In [ ]:
traces = {
            "1": {"devices": 17642,
                  "channel": "BATT",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'Battery', 
                            'ylabel': {1: 'Temperature (degC)'}
                           }, 
              options = {'frequency': '5Min'})

## Data completeness

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from scdata.tools.date import localise_date

In [ ]:
for device in test.devices:
    a = localise_date(datetime.datetime.now(), 'UTC') - device.handler.json.last_reading_at
    hours_ago = round(a.total_seconds() / 3600)
    print (device.id, device.handler.json.last_reading_at, hours_ago)
    plt.figure(figsize=(15,6))
    sns.heatmap(test.get_device(device.id).data.isna().transpose(),
                cmap="YlGnBu",
                cbar_kws={'label': 'Missing Data'}).set(title=f'{test.name}: {device.id} - ({device.handler.json.name}) {hours_ago}h')